<a href="https://colab.research.google.com/github/daisuke08253649/DeepLearning/blob/main/Defect_detection_in_cast_products.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import zipfile
import glob

def unzip_dataset(INPATH, OUTPATH):
    with zipfile.ZipFile(INPATH) as zf:
        zf.extractall(OUTPATH)

unzip_dataset(INPATH='./drive/MyDrive/DeepLearning/SIGNATE/Defect_detection_in_cast_products/train_data.zip', OUTPATH='./drive/MyDrive/DeepLearning/SIGNATE/Defect_detection_in_cast_products')

#image_name_list = os.listdir('./drive/MyDrive/DeepLearning/SIGNATE/Defect_detection_in_cast_products/train_data')
#print(image_name_list)

In [ ]:
import matplotlib.pyplot as plt
from skimage import io
import numpy as np
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, models

image_files = glob.glob('./drive/MyDrive/DeepLearning/SIGNATE/Defect_detection_in_cast_products/train_data/*')
label_datas = pd.read_csv('./drive/MyDrive/DeepLearning/SIGNATE/Defect_detection_in_cast_products/train.csv')

images = []
for file in image_files:
    img = io.imread(file)
    images.append(img)
image_datas = np.array(images)
image_datas = image_datas / 255
#print(image_datas)

labels = []
for label in label_datas.target:
    labels.append(label)
labels_lists = labels
#print(labels_dummy)


#plt.figure(figsize=(30, 30))
#for i in range(250):
    #plt.subplot(10, 25, i+1)
    #plt.title(datas.target[i])
    #plt.imshow(train_image[i])
#plt.show()

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

target_num = 2

def get_model(target_num, ispretrained=False):
    model_ft = models.resnet18(pretrained=ispretrained)
    model_ft.fc = nn.Linear(512, target_num)
    model_ft = model_ft.to(device)
    return model_ft

model = get_model(target_num, ispretrained=False)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
kf = KFold(n_splits=3, shuffle=True, random_state=32)
for fold,(train_index, val_index) in enumerate(kf.split(image_datas, labels_lists)):
    #print(f'Fold:{fold}, train_index:{len(train_index)}, val_index:{len(val_index)}')

    X_train = image_datas[train_index].transpose(0, 3, 1, 2)
    X_val = image_datas[val_index].transpose(0, 3, 1, 2)
    y_train = np.array(labels_lists)[train_index]
    y_val = np.array(labels_lists)[val_index]


    x_tr = torch.tensor(X_train, dtype=torch.float32)
    y_tr = torch.tensor(y_train, dtype=torch.long)

    x_vl = torch.tensor(X_val, dtype=torch.float32)
    y_vl = torch.tensor(y_val, dtype=torch.long)


    train_dataset = TensorDataset(x_tr, y_tr)
    val_dataset = TensorDataset(x_vl, y_vl)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)



for epoch in range(50):
    train_loss_total = 0.0

    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device).long()

        optimizer.zero_grad()
        output = model(inputs)
        train_loss = criterion(output, labels)
        train_loss.backward()
        optimizer.step()

        train_loss_total += train_loss.item() * inputs.size(0)

    train_loss_avg = train_loss_total / len(train_loader.dataset)



    val_loss_total = 0.0
    val_corrects = 0

    model.eval()
    for i, (inputs, labels) in enumerate(val_loader):
        inputs = inputs.to(device)
        labels = labels.to(device).long()

        val_output = model(inputs)
        _, val_preds = torch.max(val_output, 1)
        loss = criterion(val_output, labels)
        val_loss_total += loss.item() * inputs.size(0)
        val_corrects += torch.sum(val_preds == labels.data)

    val_loss_avg = val_loss_total / len(val_loader.dataset)
    val_corrects = val_corrects.double() / len(val_loader.dataset)

    print(f'epoch:{epoch}, train_loss:{train_loss_avg:.4f}, val_loss:{val_loss_avg:.4f}, acc:{val_corrects}')

epoch:0, train_loss:0.8926, val_loss:0.8719, acc:0.39759036144578314
epoch:1, train_loss:0.6988, val_loss:1.0319, acc:0.6024096385542169
epoch:2, train_loss:0.6329, val_loss:0.8164, acc:0.6024096385542169
epoch:3, train_loss:0.5988, val_loss:0.8325, acc:0.6024096385542169
epoch:4, train_loss:0.5890, val_loss:1.0102, acc:0.4096385542168675
epoch:5, train_loss:0.5297, val_loss:0.9925, acc:0.6144578313253012
epoch:6, train_loss:0.3784, val_loss:3.0960, acc:0.3855421686746988
epoch:7, train_loss:0.5169, val_loss:14.3174, acc:0.39759036144578314
epoch:8, train_loss:0.3519, val_loss:1.0597, acc:0.5542168674698795
epoch:9, train_loss:0.3694, val_loss:2.7846, acc:0.6024096385542169
epoch:10, train_loss:0.2437, val_loss:1.3251, acc:0.5180722891566265
epoch:11, train_loss:0.3214, val_loss:1.2941, acc:0.5060240963855422
epoch:12, train_loss:0.2024, val_loss:1.3183, acc:0.6144578313253012
epoch:13, train_loss:0.2386, val_loss:5.3399, acc:0.3855421686746988
epoch:14, train_loss:0.1823, val_loss:5.5

In [ ]:
torch.save(model.state_dict(), './drive/MyDrive/DeepLearning/SIGNATE/Defect_detection_in_cast_products/model.pth')

In [ ]:
def unzip_dataset(INPATH, OUTPATH):
    with zipfile.ZipFile(INPATH) as zf:
        zf.extractall(OUTPATH)

unzip_dataset(INPATH='./drive/MyDrive/DeepLearning/SIGNATE/Defect_detection_in_cast_products/test_data.zip', OUTPATH='./drive/MyDrive/DeepLearning/SIGNATE/Defect_detection_in_cast_products')
test_image_list = os.listdir('./drive/MyDrive/DeepLearning/SIGNATE/Defect_detection_in_cast_products/test_data')

df_test = pd.DataFrame(data=test_image_list)

df_test = df_test.rename(columns={0: 'filename'})

df_test['target'] = 0
df_test.loc[df_test['filename'].str.contains('ok'), 'target'] = 1

df_test.to_csv('sample_submission.csv', index=False)

In [ ]:
from torch.utils.data import Dataset
from PIL import Image

test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406] , [0.229, 0.224, 0.225])
])


class Test_dataset(Dataset):
    def __init__(self, transform):
        self.df = pd.read_csv('./drive/MyDrive/DeepLearning/SIGNATE/Defect_detection_in_cast_products/sample_submission.csv')
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        file = self.df['filename'][index]
        image = Image.open('./drive/MyDrive/DeepLearning/SIGNATE/Defect_detection_in_cast_products/test_data/' + file)
        image = self.transform(image)

        return image, file

test_dataset = Test_dataset(test_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
def get_model():
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(512, 2)
    model = model.to(device)
    return model

model = get_model()
model.load_state_dict(torch.load('./drive/MyDrive/DeepLearning/SIGNATE/Defect_detection_in_cast_products/model.pth'))

pred = []
for i, (inputs, labels) in enumerate(test_dataloader):
    model.eval()

    inputs = inputs.to(device)

    output = model(inputs)
    _, preds = torch.max(output, 1)

    pred.extend(preds.cpu().numpy().tolist())

df_test['target'] = pred

data_csv = './drive/MyDrive/DeepLearning/SIGNATE/Defect_detection_in_cast_products/sample_submission.csv'

with open(data_csv, 'r') as f:
    lines = f.readlines()

lines.pop(0)

df_test.to_csv(data_csv, index=False, header=False, mode='w')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
